In [4]:
from adult_read import *

In [6]:
adult

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,salary,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,...,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia,sex_ Female,sex_ Male
0,0.301370,0.044302,0.800000,0.021740,0.0,0.397959,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,0.452055,0.048238,0.800000,0.000000,0.0,0.122449,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,0.287671,0.138113,0.533333,0.000000,0.0,0.397959,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,0.493151,0.151068,0.400000,0.000000,0.0,0.397959,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,0.150685,0.221488,0.800000,0.000000,0.0,0.397959,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0.136986,0.166404,0.733333,0.000000,0.0,0.377551,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
32557,0.315068,0.096500,0.533333,0.000000,0.0,0.397959,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
32558,0.561644,0.094827,0.533333,0.000000,0.0,0.397959,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
32559,0.068493,0.128499,0.533333,0.000000,0.0,0.193878,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
